# Data Cleaning and Exploration

#### Purpose
- Use Earthquake API to get Earthquake data used for visualization and analysis
- Manipulate JSON response into a usable dataframe that can be exported out into csv
- Sort by Magnitude to identify Top 5 Earthquakes which will be our focus for visualizations page

#### API
https://earthquake.usgs.gov/fdsnws/event/1/

#### Parameters
- Format: geojson
- Start Time: 2000-01-01
- End Time: Present (default)
- Minimum Magnitude: 7

In [1]:
# Dependencies and Setup
import json
import matplotlib.pyplot as plt
import pandas as pd
from pandas import json_normalize
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint 
import datetime as dt

In [2]:
# SQL Alchemy Dependencies

import sqlalchemy
from sqlalchemy import create_engine

In [3]:
# Perform API Call to get json data for earthquakes from 2000 to current
url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2000-01-01&minmagnitude=7"
response = requests.get(url).json()

In [4]:
print(response)

{'type': 'FeatureCollection', 'metadata': {'generated': 1618542668000, 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2000-01-01&minmagnitude=7', 'title': 'USGS Earthquakes', 'status': 200, 'api': '1.10.3', 'count': 320}, 'features': [{'type': 'Feature', 'properties': {'mag': 7, 'place': '29 km ENE of Ishinomaki, Japan', 'time': 1616231384002, 'updated': 1618395876097, 'tz': None, 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/us7000dl6y', 'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us7000dl6y&format=geojson', 'felt': 408, 'cdi': 7.3, 'mmi': 7.008, 'alert': 'yellow', 'status': 'reviewed', 'tsunami': 1, 'sig': 1052, 'net': 'us', 'code': '7000dl6y', 'ids': ',at00qq9g49,us7000dl6y,usauto7000dl6y,pt21079003,', 'sources': ',at,us,usauto,pt,', 'types': ',associate,dyfi,finite-fault,general-text,ground-failure,impact-link,impact-text,internal-moment-tensor,internal-origin,losspager,moment-tensor,origin,phase-data,shakemap,'

In [5]:
#commented it out because it was a lot of information
#pprint(response)

In [6]:
##testing indices within geo-json
time=response['features'][0]['properties']['time']
lat=response['features'][0]['geometry']['coordinates'][0]
long=response['features'][0]['geometry']['coordinates'][1]
alrt=response['features'][0]['properties']['alert']
mag= response['features'][0]['properties']['mag']
tsu=response['features'][0]['properties']['tsunami']
place_test=response['features'][0]['properties']['place']
type_test=response['features'][0]['properties']['type']
time

1616231384002

In [7]:
# List out desired fields that will make up Dataframe
time = []
latitude = []
longitude = []
alert = []
depth = []
magnitude = []
tsunami = []
place = []
type = []

In [8]:
# Create blank data frame 
quake_df = pd.DataFrame(columns = ['Time', 'Longitude', 'Latitude', 'Alert','Depth','Magnitude', 'Tsunami','Place','Type'])

In [9]:
quake_df

,Time,Longitude,Latitude,Alert,Depth,Magnitude,Tsunami,Place,Type


In [10]:
for i in range(len(response['features'])): 
    time.append(response['features'][i]['properties']['time'])
    latitude.append(response['features'][i]['geometry']['coordinates'][0])
    longitude.append(response['features'][i]['geometry']['coordinates'][1])
    depth.append(response['features'][i]['geometry']['coordinates'][2])
    alert.append(response['features'][i]['properties']['alert'])
    magnitude.append(response['features'][i]['properties']['mag'])
    tsunami.append(response['features'][i]['properties']['tsunami'])
    place.append(response['features'][i]['properties']['place'])
    type.append(response['features'][i]['properties']['type'])

In [11]:
### Confirming length of lists
print(len(latitude))
print(len(longitude))
print(len(time))
print(len(alert))
print(len(magnitude))
print(len(tsunami))
print(len(place))
print(len(type))
print(len(depth))

320
320
320
320
320
320
320
320
320


In [12]:
##Assigning list to column values
quake_df['Time']=time
quake_df['Latitude']=latitude
quake_df['Alert']=alert
quake_df['Magnitude']=magnitude
quake_df['Tsunami']=tsunami
quake_df['Place']=place
quake_df['Type']=type
quake_df['Longitude']=longitude
quake_df['Depth']=depth

In [13]:
#convert unix time code to human readable time
quake_df['Converted_Time_GMT'] = pd.to_datetime(quake_df['Time'],unit='ms').dt.date

In [14]:
quake_df

,Time,Longitude,Latitude,Alert,Depth,Magnitude,Tsunami,Place,Type,Converted_Time_GMT
0,1616231384002,38.4754,141.6328,yellow,43.00,7.0,1,"29 km ENE of Ishinomaki, Japan",earthquake,2021-03-20
1,1614886113178,-29.7228,-177.2794,green,28.93,8.1,1,"Kermadec Islands, New Zealand",earthquake,2021-03-04
2,1614879683370,-29.6768,-177.8398,green,43.00,7.4,1,"Kermadec Islands, New Zealand",earthquake,2021-03-04
3,1614864454885,-37.5459,179.3587,green,10.00,7.3,1,"170 km NE of Gisborne, New Zealand",earthquake,2021-03-04
4,1613225270194,37.7534,141.7145,orange,44.07,7.1,1,"69 km ENE of Namie, Japan",earthquake,2021-02-13
...,...,...,...,...,...,...,...,...,...,...
315,957414076210,-1.1050,123.5730,None,26.00,7.6,0,"Sulawesi, Indonesia",earthquake,2000-05-04
316,956482043320,-28.3070,-62.9900,None,608.50,7.0,0,"Santiago Del Estero, Argentina",earthquake,2000-04-23
317,954241222510,22.3380,143.7300,None,126.50,7.6,0,"Volcano Islands, Japan region",earthquake,2000-03-28
318,951443038640,-19.5280,173.8180,None,33.00,7.1,0,Vanuatu region,earthquake,2000-02-25


In [15]:
#checking data types
quake_df.dtypes

Time                    int64
Longitude             float64
Latitude              float64
Alert                  object
Depth                 float64
Magnitude             float64
Tsunami                 int64
Place                  object
Type                   object
Converted_Time_GMT     object
dtype: object

In [16]:
#dropping tsunami
quake_df = quake_df.drop(columns='Tsunami')

In [17]:
quake_df

,Time,Longitude,Latitude,Alert,Depth,Magnitude,Place,Type,Converted_Time_GMT
0,1616231384002,38.4754,141.6328,yellow,43.00,7.0,"29 km ENE of Ishinomaki, Japan",earthquake,2021-03-20
1,1614886113178,-29.7228,-177.2794,green,28.93,8.1,"Kermadec Islands, New Zealand",earthquake,2021-03-04
2,1614879683370,-29.6768,-177.8398,green,43.00,7.4,"Kermadec Islands, New Zealand",earthquake,2021-03-04
3,1614864454885,-37.5459,179.3587,green,10.00,7.3,"170 km NE of Gisborne, New Zealand",earthquake,2021-03-04
4,1613225270194,37.7534,141.7145,orange,44.07,7.1,"69 km ENE of Namie, Japan",earthquake,2021-02-13
...,...,...,...,...,...,...,...,...,...
315,957414076210,-1.1050,123.5730,None,26.00,7.6,"Sulawesi, Indonesia",earthquake,2000-05-04
316,956482043320,-28.3070,-62.9900,None,608.50,7.0,"Santiago Del Estero, Argentina",earthquake,2000-04-23
317,954241222510,22.3380,143.7300,None,126.50,7.6,"Volcano Islands, Japan region",earthquake,2000-03-28
318,951443038640,-19.5280,173.8180,None,33.00,7.1,Vanuatu region,earthquake,2000-02-25


In [18]:
#order by magnitude
quake_df = quake_df.sort_values(by='Magnitude', ascending = False)

In [19]:
quake_df

,Time,Longitude,Latitude,Alert,Depth,Magnitude,Place,Type,Converted_Time_GMT
246,1104022733450,3.2950,95.9820,None,30.0,9.1,2004 Sumatra - Andaman Islands Earthquake,earthquake,2004-12-26
146,1299822384120,38.2970,142.3730,None,29.0,9.1,"2011 Great Tohoku Earthquake, Japan",earthquake,2011-03-11
172,1267252451530,-36.1220,-72.8980,None,22.9,8.8,"offshore Bio-Bio, Chile",earthquake,2010-02-27
242,1112026176530,2.0850,97.1080,None,30.0,8.6,"northern Sumatra, Indonesia",earthquake,2005-03-28
127,1334133516720,2.3270,93.0630,yellow,20.0,8.6,off the west coast of northern Sumatra,earthquake,2012-04-11
...,...,...,...,...,...,...,...,...,...
248,1101666734130,43.0060,145.1190,None,39.0,7.0,"Hokkaido, Japan region",earthquake,2004-11-28
103,1377879902300,51.5370,-175.2297,green,29.0,7.0,"101km SW of Atka, Alaska",earthquake,2013-08-30
110,1365223355860,-3.5170,138.4760,green,66.0,7.0,"240km E of Enarotali, Indonesia",earthquake,2013-04-06
112,1360115654610,-10.4990,165.5880,green,8.8,7.0,"27km NNW of Lata, Solomon Islands",earthquake,2013-02-06


In [24]:
# Read in GDP_data.csv
gdp_df = pd.read_csv("GDP_data.csv")
gdp_df.head()

,Series Name,Series Code,Country Name,Country Code,2000,2001,2002,2003,2004,2005,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,Japan,JPN,2.779633,0.406336,0.117993,1.528220,2.204688,1.662670,...,-0.115421,1.495090,2.000268,0.374719,1.222921,0.521944,2.168291,0.323207,0.654152,..
1,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,Indonesia,IDN,4.920068,3.643466,4.499475,4.780369,5.030874,5.692571,...,6.169784,6.030051,5.557264,5.006668,4.876322,5.033069,5.069786,5.169706,5.024714,..
2,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,Chile,CHL,5.326938,3.303047,3.106971,4.091048,7.209540,5.742830,...,6.110919,5.318628,4.045004,1.766740,2.303767,1.711089,1.188573,3.949321,1.054443,..
3,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,China,CHN,8.490093,8.335733,9.133631,10.038030,10.113621,11.394592,...,9.550832,7.863736,7.766150,7.425764,7.041329,6.848762,6.947201,6.749774,5.949714,..
4,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,United States,USA,4.127484,0.998341,1.741695,2.861211,3.798891,3.513214,...,1.550836,2.249546,1.842081,2.525973,2.908022,1.637838,2.369801,2.927326,2.161176,..


In [25]:
#Writing out to csv
quake_df.to_csv('quake_df.csv')

In [28]:
# Create database connection and load quake.df into database
rds_connection_string = "postgres:postgres@localhost:5432/earthquake_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [29]:
# Push quake_df to earthquake_db
quake_df.to_sql(name='quake', con=engine, if_exists='replace', index=False)

In [30]:
# Push gdp_df to earthquake_db
gdp_df.to_sql(name='gdp',con=engine, if_exists='replace', index=False)